In [4]:
# This notebook has version issue
# If you don't downgrade torch and torchvision, you can get runtime cuda error.
# ex> "runtimeerror: gather_out_cuda(): expected dtype int64 for index"

# !pip install torch==1.5.0
# !pip install torchvision==0.6.0

In [5]:
# !pip install torch==1.5.0
# !pip install torchvision==0.6.0

In [6]:
import os

import sys
sys.path.insert(0, '/home/kerrykim/PycharmProjects/3.road_obstacle_detection/effdet')
sys.path.insert(0, '/home/kerrykim/PycharmProjects/3.road_obstacle_detection/omegaconf')
sys.path.insert(0, '/home/kerrykim/PycharmProjects/3.road_obstacle_detection/wbf')
                
from wbf.ensemble_boxes import *
import torch
import numpy as np
import pandas as pd
from glob import glob
from torch.utils.data import Dataset,DataLoader
import albumentations as A
from albumentations.pytorch.transforms import ToTensorV2
import cv2
import gc
from matplotlib import pyplot as plt
from effdet import get_efficientdet_config, EfficientDet, DetBenchEval
from effdet.efficientdet import HeadNet

In [7]:
## Configurations
class CFG:
    mode = 'test'
    seed = 42
    print_freq = 30

    n_class = 13

    img_x = 512
    img_y = 512

    num_fold = 5
    num_epoch = 50
    batch_size = 2
    num_workers = 2    # decide how many data upload to dataset for a batch
                       # if n_workers 2, dataloader works twice for a batch.
                       # It has impact for cuda memory too

    lr = 0.0002

    max_grad_norm = 1000


    data_dir = './test'
    ckpt_dir = './checkpoint/final_f1_ep3_bt8_date11.11-14:13.pth'
    result_dir = './result'
    log_dir = './log'
    
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

In [8]:
class Dataset(Dataset):
    def __init__(self, data_dir, transform=None):
        self.data_dir = data_dir
        self.transform = transform

        lst_input = os.listdir(self.data_dir)
        self.lst_input = lst_input
        # self.image_id = [image_id.split('/')[-1][:-4] for image_id in lst_input]

    def __len__(self):
        return len(self.lst_input)

    def __getitem__(self, index):
        image_id = self.lst_input[index]
        image = cv2.imread(os.path.join(self.data_dir, self.lst_input[index]), cv2.IMREAD_COLOR)
        image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB).astype(np.float32)
        image /= 255.0

        if self.transform:
            sample = {'image' : image}
            sample = self.transform(**sample)    # 이때까지 해왔던 입력방식은 cv2 이미지를 직접 입력, 여기선 numpy 로 입력
            image = sample['image']

        return image, image_id

In [17]:
def load_net(ckpt_path):
    config = get_efficientdet_config('tf_efficientdet_d5')
    net = EfficientDet(config, pretrained_backbone=False)

    config.num_classes = 1
    config.image_size= 512
    net.class_net = HeadNet(config, num_outputs=config.num_classes, norm_kwargs=dict(eps=.001, momentum=.01))

    checkpoint = torch.load(ckpt_path)
    # net.load_state_dict(checkpoint['model_state_dict'])
    # net.load_state_dict(checkpoint, strict=False) # strict=False 해주니까 된다.
    net.load_state_dict(checkpoint, strict=False)
    
    del checkpoint
    gc.collect()

    net = (DetBenchEval(net, config)).to(device)

    return net

In [18]:
def transform_test():
    return A.Compose([A.Resize(height=512, width=512, p=1.0), ToTensorV2(p=1.0)], p=1.0)

In [19]:
def test_one_epoch(loader_test, net, score_threshold=0.22):
    net.eval()
    pred = []

    ##
    for batch, (images, image_ids) in enumerate(loader_test, 1):
        images = torch.stack(images)
        images = images.to(device).float()

        with torch.no_grad():
            det = net(images, torch.tensor([1]*images.shape[0]).float().cuda())
            output = []
            for i in range(images.shape[0]):
                boxes = det[i].detach().cpu().numpy()[:,:4]
                scores = det[i].detach().cpu().numpy()[:,4]
                indexes = np.where(scores > score_threshold)[0]
                boxes = boxes[indexes]
                boxes[:, 2] = boxes[:, 2] + boxes[:, 0]
                boxes[:, 3] = boxes[:, 3] + boxes[:, 1]
                output.append({
                    'boxes': boxes[indexes],
                    'scores': scores[indexes],
                })

            output = [output]

        for i, image in enumerate(images):
            boxes, scores, labels = run_wbf(output, image_index=i)
            boxes = (boxes * 2).astype(np.int32).clip(min=0, max=1023)
            image_id = image_ids[i]

            boxes[:, 2] = boxes[:, 2] - boxes[:, 0]
            boxes[:, 3] = boxes[:, 3] - boxes[:, 1]

            tmp = {'image_id': image_id, 'PredictionString': format_prediction_string(boxes, scores)}

            pred.append(tmp)

    return pred

In [20]:
def run_wbf(predictions, image_index, image_size=512, iou_thr=0.44, skip_box_thr=0.43, weights=None):
    boxes = [(prediction[image_index]['boxes']/(image_size-1)).tolist()  for prediction in predictions]
    scores = [prediction[image_index]['scores'].tolist()  for prediction in predictions]
    labels = [np.ones(prediction[image_index]['scores'].shape[0]).tolist() for prediction in predictions]
    boxes, scores, labels = weighted_boxes_fusion(boxes, scores, labels, weights=None, iou_thr=iou_thr, skip_box_thr=skip_box_thr)
    boxes = boxes*(image_size-1)
    return boxes, scores, labels

In [21]:
def format_prediction_string(boxes, scores):
    pred_strings = []
    for j in zip(scores, boxes):
        pred_strings.append("{0:.4f} {1} {2} {3} {4}".format(j[0], j[1][0], j[1][1], j[1][2], j[1][3]))
    return " ".join(pred_strings)

In [22]:
def collate_fn(batch):
    return tuple(zip(*batch))

In [23]:
def test():
    dataset_test = Dataset(data_dir=CFG.data_dir, transform=transform_test())
    loader_test = DataLoader(dataset_test, batch_size=CFG.batch_size, shuffle=False, num_workers=8, drop_last=False, collate_fn=collate_fn, pin_memory=True)

    net = load_net(CFG.ckpt_dir)
    pred = test_one_epoch(loader_test, net)

    submission = pd.DataFrame(pred, columns=['image_id', 'PredictionString'])
    submission.to_csv('submission.csv',index=False)
    # submission.to_csv(os.path.join(CFG.result_dir, 'submission.csv'))
    
    return submission


In [24]:
##
if __name__ == "__main__":
    if CFG.mode == "test":
        submission = test()

/home/kerrykim/anaconda3/envs/skku/lib/python3.9/site-packages/torch/nn/functional.py:766: UserWarning: Note that order of the arguments: ceil_mode and return_indices will changeto match the args list in nn.MaxPool2d in a future release.
  warnings.warn("Note that order of the arguments: ceil_mode and return_indices will change"


RuntimeError: gather(): Expected dtype int64 for index

In [ ]:
submission

In [ ]:
torch.cuda.empty_cache()
gc.collect()